# Data Preparation

In [5]:
import pandas as pd
df1 = pd.read_csv("../data/repo_function_matched_v3_1.csv")
df2 = pd.read_csv("../data/repo_function_matched_v3_2.csv")
df3 = pd.read_csv("../data/repo_function_matched_v4.csv")
#df4 = pd.read_csv("../data/repo_function_matched_v4_2.csv")
df = df3.append(df2, ignore_index = True)
df = df.append(df1, ignore_index = True)
df = df.drop_duplicates()

In [2]:
len(df)

543451

In [3]:
df.head()

,line,function,package,class,repo,path,author,commit,code
0,39,enable_eager_execution,tensorflow,NaN,tensorflow,tensorflow/contrib/eager/python/examples/noteb...,blamb@google.com,486b96a51d6b0b394edf77d182f7283a8ec03e0d,tf.enable_eager_execution()
1,54,square,tensorflow,NaN,tensorflow,tensorflow/contrib/eager/python/examples/noteb...,blamb@google.com,486b96a51d6b0b394edf77d182f7283a8ec03e0d,return tf.square(tf.sin(x))
2,54,sin,tensorflow,NaN,tensorflow,tensorflow/contrib/eager/python/examples/noteb...,blamb@google.com,486b96a51d6b0b394edf77d182f7283a8ec03e0d,return tf.square(tf.sin(x))
3,56,f,NaN,NaN,tensorflow,tensorflow/contrib/eager/python/examples/noteb...,blamb@google.com,486b96a51d6b0b394edf77d182f7283a8ec03e0d,assert f(pi/2).numpy() == 1.0
4,63,numpy,tensorflow,NaN,tensorflow,tensorflow/contrib/eager/python/examples/noteb...,blamb@google.com,486b96a51d6b0b394edf77d182f7283a8ec03e0d,assert tf.abs(grad_f(pi/2)[0]).numpy() < 1e-7


In [8]:
df.isna().sum()

line             0
function         2
package     305752
class       489045
repo             0
path             0
author         315
commit           0
code            37
dtype: int64

In [6]:
clean = df.drop(df[df.author.isnull()].index)
clean = clean.drop(clean[clean.function.isnull()].index)

In [14]:
clean.isna().sum()

line             0
function         0
package     305658
class       488734
repo             0
path             0
author           0
commit           0
code            37
dtype: int64

In [7]:
import numpy as np
def combining(func, pack, c):
    name = func
    if c is not np.nan:
        name = ".".join([c, func])
    if pack is not np.nan:
        name = ".".join([pack, name])
    return name

In [8]:
clean['combined'] = clean.apply(lambda x: combining(x['function'], x['package'], x['class']), axis = 1)

In [31]:
clean.head()

,line,function,package,class,repo,path,author,commit,code,combined
0,39,enable_eager_execution,tensorflow,NaN,tensorflow,tensorflow/contrib/eager/python/examples/noteb...,blamb@google.com,486b96a51d6b0b394edf77d182f7283a8ec03e0d,tf.enable_eager_execution(),tensorflow.enable_eager_execution
1,54,square,tensorflow,NaN,tensorflow,tensorflow/contrib/eager/python/examples/noteb...,blamb@google.com,486b96a51d6b0b394edf77d182f7283a8ec03e0d,return tf.square(tf.sin(x)),tensorflow.square
2,54,sin,tensorflow,NaN,tensorflow,tensorflow/contrib/eager/python/examples/noteb...,blamb@google.com,486b96a51d6b0b394edf77d182f7283a8ec03e0d,return tf.square(tf.sin(x)),tensorflow.sin
3,56,f,NaN,NaN,tensorflow,tensorflow/contrib/eager/python/examples/noteb...,blamb@google.com,486b96a51d6b0b394edf77d182f7283a8ec03e0d,assert f(pi/2).numpy() == 1.0,f
4,63,numpy,tensorflow,NaN,tensorflow,tensorflow/contrib/eager/python/examples/noteb...,blamb@google.com,486b96a51d6b0b394edf77d182f7283a8ec03e0d,assert tf.abs(grad_f(pi/2)[0]).numpy() < 1e-7,tensorflow.numpy


In [9]:
authors = clean[['author','combined']].groupby(['author'])['combined'].apply(list).reset_index()

In [34]:
len(authors)

5428

In [10]:
authors.head()

,author,combined
0,--global,"[get_ipython, pandas.read_csv, pandas.set_inde..."
1,09zabeli@gmail.com,"[get_ipython, numpy.all, numpy.all, exp, numpy..."
2,0u812@users.noreply.github.com,"[tellurium.getBoundarySpeciesIds, tellurium.ge..."
3,1021036214@qq.com,"[int, input, int, input, int, input, print, ra..."
4,1070251317@qq.com,"[range, print, print, input, int, input, print..."


# Topic Modelling

In [1]:
#Import all necessary libraries
import glob
import pandas as pd
import numpy as np

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
warnings.filterwarnings('ignore')
import _pickle as pickle

import re
import random
import collections

from gensim import corpora, models
import gensim

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from scipy import stats

pyLDAvis.enable_notebook()
random.seed(1234)

from sklearn.metrics.cluster import adjusted_rand_score
pd.options.display.max_columns = None

c:\users\patrick\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [20]:
%%time
#build gensim corpus
texts = authors['combined']
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=10, no_above=0.4)
dictionary.compactify()
corpus = [dictionary.doc2bow(text) for text in texts]

Wall time: 1.03 s


In [19]:
len(dictionary)

2682

In [27]:
%%time
ldamodelnormal = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=10, chunksize=100, update_every=0, alpha=1/15, random_state=1)

Wall time: 36.1 s


In [21]:
# helper to create descriptive tables (doc-topic probabilities) and visualizations for LDA models 
def getModelResults(ldamodel, corpus, dictionary):
    vis = pyLDAvis.gensim.prepare(ldamodel,corpus, dictionary, sort_topics=False)
    transformed = ldamodel.get_document_topics(corpus)
    df = pd.DataFrame.from_records([{v:k for v, k in row} for row in transformed])
    return vis, df    

In [28]:
%%time
# get descriptive stuff for all models
normalv, dfnormal = getModelResults(ldamodelnormal, corpus, dictionary)

Wall time: 6.27 s


In [29]:
#print regular topics (top word probabilities)
ldamodelnormal.print_topics(num_words=8)

[(0,
  '0.040*"len" + 0.035*"range" + 0.025*"numpy.linspace" + 0.025*"numpy.array" + 0.021*"plot" + 0.018*"matplotlib.pyplot.plot" + 0.018*"matplotlib.pyplot.subplots" + 0.017*"numpy.arange"'),
 (1,
  '0.093*"int" + 0.083*"input" + 0.064*"range" + 0.034*"len" + 0.020*"dict" + 0.017*"requests.get" + 0.016*"run" + 0.015*"list"'),
 (2,
  '0.056*"len" + 0.046*"range" + 0.031*"open" + 0.029*"pandas.read_csv" + 0.015*"matplotlib.pyplot.plot" + 0.015*"str" + 0.014*"list" + 0.014*"pandas.DataFrame"'),
 (3,
  '0.062*"len" + 0.048*"range" + 0.034*"str" + 0.030*"add" + 0.022*"Sequential.add" + 0.021*"matplotlib.pyplot.plot" + 0.018*"list" + 0.018*"show_doc"'),
 (4,
  '0.063*"matplotlib.pyplot.plot" + 0.044*"numpy.array" + 0.033*"matplotlib.pyplot.figure" + 0.031*"matplotlib.pyplot.title" + 0.030*"matplotlib.pyplot.ylabel" + 0.030*"matplotlib.pyplot.xlabel" + 0.027*"range" + 0.026*"matplotlib.pyplot.show"')]

In [30]:
# inspect regular topics - ATTENTION: ALL TOPICS ARE SHIFTED WITH ID +1 w.r.t. GENSIM
normalv

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.068161  0.004449  1       1        20.972576
1      0.171291  0.055490  2       1        12.313975
2      0.010759 -0.121611  3       1        24.230827
3      0.014392  0.000126  4       1        16.208269
4     -0.128280  0.061546  5       1        26.274351, topic_info=     Category          Freq                           Term         Total  \
term                                                                       
27    Default  3459.000000   input                          3459.000000    
16    Default  5669.000000   int                            5669.000000    
79    Default  1768.000000   add                            1768.000000    
1102  Default  1778.000000   datetime.datetime              1778.000000    
20    Default  12329.000000  range                          12329.000000   
28    Default  12269.000000  len                            12269.000000   
376   Default  1336.000000   Sequential.add                 1336.000000    
194   Default  869.000000    show_doc                       869.000000     
146   Default  8345.000000   matplotlib.pyplot.plot         8345.000000    
70    Default  1010.000000   requests.get                   1010.000000    
73    Default  895.000000    run                            895.000000     
157   Default  3542.000000   matplotlib.pyplot.xlabel       3542.000000    
44    Default  4564.000000   str                            4564.000000    
147   Default  3585.000000   matplotlib.pyplot.title        3585.000000    
158   Default  3531.000000   matplotlib.pyplot.ylabel       3531.000000    
71    Default  729.000000    requests.json                  729.000000     
161   Default  6476.000000   numpy.array                    6476.000000    
19    Default  3552.000000   open                           3552.000000    
123   Default  2154.000000   dict                           2154.000000    
6     Default  3728.000000   pandas.read_csv                3728.000000    
106   Default  1171.000000   display                        1171.000000    
124   Default  3186.000000   numpy.linspace                 3186.000000    
429   Default  1109.000000   tensorflow.Variable            1109.000000    
281   Default  4501.000000   matplotlib.pyplot.figure       4501.000000    
38    Default  556.000000    random.randint                 556.000000     
436   Default  1091.000000   tensorflow.placeholder         1091.000000    
366   Default  1846.000000   matplotlib.pyplot.subplot      1846.000000    
327   Default  829.000000    pyplot.plot                    829.000000     
164   Default  1329.000000   numpy.exp                      1329.000000    
137   Default  568.000000    join                           568.000000     
...       ...          ...         ...                              ...    
146   Topic5   4939.711914   matplotlib.pyplot.plot         8345.558594    
139   Topic5   724.000000    Image                          1109.750000    
281   Topic5   2599.596680   matplotlib.pyplot.figure       4501.426270    
208   Topic5   800.653137    matplotlib.pyplot.ylim         1259.782959    
161   Topic5   3470.468262   numpy.array                    6476.413574    
735   Topic5   366.095306    matplotlib.pyplot.gca          537.520630     
145   Topic5   1326.803467   matplotlib.pyplot.legend       2362.352295    
144   Topic5   1166.957153   matplotlib.pyplot.imshow       2062.647705    
282   Topic5   879.967834    matplotlib.pyplot.scatter      1551.785645    
156   Topic5   2067.798828   matplotlib.pyplot.show         4483.295898    
154   Topic5   403.110291    matplotlib.pyplot.axis         667.937317     
168   Topic5   502.221191    numpy.ones                     901.711487     
13    Topic5   1427.022217   numpy.arange                   3275.033203    
364   Topic5   445.106995    matplotlib.pyplot.colorbar     794.026855     
229   T

In [31]:
#inspect first 20 documents and their topic distributions
pd.set_option('display.max_colwidth', -1)
topics_post = pd.concat([authors['author'], dfnormal], axis=1)
topics_post.head(10)

,author,0,1,2,3,4
0,--global,NaN,NaN,0.981395,NaN,NaN
1,09zabeli@gmail.com,NaN,0.545563,NaN,NaN,0.438221
2,0u812@users.noreply.github.com,NaN,NaN,0.986207,NaN,NaN
3,1021036214@qq.com,NaN,0.996800,NaN,NaN,NaN
4,1070251317@qq.com,NaN,0.996911,NaN,NaN,NaN
5,1098035510@qq.com,NaN,0.999100,NaN,NaN,NaN
6,1109253641np@sina.com,NaN,0.996875,NaN,NaN,NaN
7,1119062521@qq.com,NaN,0.998779,NaN,NaN,NaN
8,1147619850@qq.com,NaN,0.981395,NaN,NaN,NaN
9,1150596359@qq.com,NaN,0.989873,NaN,NaN,NaN
